Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: - - .

In [110]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

In [102]:
KEYWORDS = ['python', 'парсинг']
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html.parser')
news_blocks = soup.find_all('article', class_='post')
articles_intro = list(map(lambda x: x.find('h2', class_='post__title'), news_blocks))
a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
kom_news = pd.DataFrame()
for link in a_list:
    soup_1 = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    #hub  = requests.get(link).text.lower()
    #print(title_element.text, title_element.attrs.get('href'))
    hub_text = soup_1.find("div", class_='post__text').text
    if any([desired in hub_text for desired in KEYWORDS]):
        #hub_text = soup_1.find("div", class_='post__text').text
        title = soup_1.find('span', class_='post__title-text').text
        date = soup_1.find('span', class_='post__time').text
        author = soup_1.find('span', class_='user-info__nickname').text
        row = {'date': date, 'author': author, 'title': title, 'hub_text': hub_text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])
kom_news       

,date,author,title,hub_text
0,сегодня в 12:48,DigitalEcosystems,Дайджест интересных материалов для мобильного ...,"В этом дайджесте — обновления Android, собстве..."
0,вчера в 22:41,bymytry,Токсичность Белорусского IT аутсорса,ПредисловиеЯ прекрасно понимаю что описываемые...


Задание 2.
Обязательная часть¶
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>.

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [171]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Version':'0', 'Vaar-Header-App-Product':'hackcheck-web-avast'}
params = {'emailAddresses': EMAIL }
leak_info = pd.DataFrame()
req = requests.post(url = URL, data = json.dumps(params), headers=headers)
req.text
list_ = json.loads(req.text)
list_
for info in list_['breaches']:
    site = list_['breaches'][info]['site']
    publishDate = list_['breaches'][info]['publishDate']
    description = list_['breaches'][info]['description']
    row = {'publishDate': publishDate, 'site': site, 'description': description}
    leak_info = pd.concat([leak_info, pd.DataFrame([row])])
leak_info

,publishDate,site,description
0,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."
0,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
0,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
0,2017-12-01T00:00:00Z,creocommunity.com,"At an unconfirmed date, Creo Community's user ..."
0,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
